In [69]:
import numpy as np
import  functools
import pandas as pd

In [ ]:
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('expand_frame_repr', False)  # 即“禁止换行”
# pd.set_option('display.precision', 2) #展示两位小数点
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df_data_交易=pd.read_csv('data/账户交易信息.csv')
df_data_交易.columns = ['交易流水序号','账户代号','对方账号','借贷标志','交易金额','交易余额','对方行号','交易日期','交易时间','交易渠道','摘要代号','对方名称长度']
df_data_交易 = df_data_交易.sort_values(["交易日期" , "交易时间"])
df_data_账户=pd.read_csv('data/账户静态信息.csv')
df_data_账户.columns = ['账户代号','开户日期','开户行代号','客户性别','年龄']
df_label_train=pd.read_csv('data/训练集标签.csv')
df_label_train.columns = ['账户代号','label']
df_label_test=pd.read_csv('data/test_dataset.csv')
df_label_test.columns = ['账户代号']## 读取数据
df_label_all=pd.concat([df_label_train,df_label_test])

In [ ]:
list_交易_金额为负_index = list(df_data_交易[df_data_交易["交易金额"] < 0].index)
list_mz_index = [] ;
list_交易_金额为负_对应的失败交易_index = [] ;
for i in list_交易_金额为负_index :
    S_now = df_data_交易.loc[i] ;
    j = i ;
    while (True) :
        j -= 1 ;
        S_nex = df_data_交易.loc[j] ;
        if (S_now["交易金额"] == -S_nex["交易金额"]) :
            if (S_now["账户代号"] == S_nex["账户代号"]) :
                list_交易_金额为负_对应的失败交易_index.append(j) ;
                list_mz_index.append(i) ;
                break ;
list_交易_抹账需要去除_index = list_交易_金额为负_index + list_交易_金额为负_对应的失败交易_index ;
df_data_交易_去除抹账后=df_data_交易[~df_data_交易.index.isin(list_交易_抹账需要去除_index)]

In [ ]:
df_data_交易_去除抹账后.shape

In [104]:
df_data_交易_去除抹账后[df_data_交易_去除抹账后["交易金额"] == 520]

,交易流水序号,账户代号,对方账号,借贷标志,交易金额,交易余额,对方行号,交易日期,交易时间,交易渠道,摘要代号,对方名称长度,label
757,310D1CFAD7C8222FE5F53585466F76F4,D18A63ED9D7D96A5,EDE31EF0F69DC342,0,520.00,4967.32,469F7281,2020-03-01,06:51:24,E96ED478,4E0CB6FB,12,NaN
3397,227F4ACB7A1D1C627E4E70616136EB08,629B0635E6C7C673,EDE31EF0F69DC342,0,520.00,1260.09,469F7281,2020-03-01,11:45:11,E96ED478,4E0CB6FB,12,NaN
4303,5CE190CA519D07C6BD2456F9655F847D,C0C793B151DF33CE,EDE31EF0F69DC342,0,520.00,2575.13,469F7281,2020-03-01,13:04:44,E96ED478,4E0CB6FB,12,NaN
4331,C89487717EA57F379B033373F24C8EE8,C0C793B151DF33CE,EDE31EF0F69DC342,0,520.00,741.13,469F7281,2020-03-01,13:07:05,E96ED478,4E0CB6FB,12,NaN
4438,16978DD949715CC476C45A3BEF52BBBC,C0C793B151DF33CE,EDE31EF0F69DC342,0,520.00,221.13,469F7281,2020-03-01,13:19:35,E96ED478,4E0CB6FB,12,NaN
5041,54450E03F1CF9DDE0BCDB113F9A652CA,1A47BD90D642C3D0,03074BB6EBB18A55,1,520.00,5270.30,CAD56B44,2020-03-01,14:16:50,621461AF,A3C65C29,6,NaN
7671,42F27C89AA921C45BD31FE5B903BF26E,DA2B51367E5A9340,DCAD1BBD38B41458,1,520.00,528.89,98CB458A,2020-03-01,18:35:11,621461AF,A3C65C29,9,NaN
7878,5392FDC9B746BDB2788CCEF05841E497,A1429D4FCC9DFA55,642FF3A44F7DE490,0,520.00,14.31,9645A0D0,2020-03-01,18:56:47,091D584F,A97DA629,9,NaN
8912,2D34626A0AD27101E277A940C930A032,976CEA39568207DF,EDE31EF0F69DC342,0,520.00,250.13,469F7281,2020-03-01,20:57:45,E96ED478,4E0CB6FB,12,NaN
9972,077AAA27486CB8CCE5CC6F44ED7E743C,15F1DC4DCEF6630D,EDE31EF0F69DC342,0,520.00,51327.46,469F7281,2020-03-02,00:01:15,E96ED478,4E0CB6FB,12,NaN
